# Weeks 13 and 14: Capstone Project Part 6

## Objective:
Submit a fully integrated prototype that supports the following:
1. Conversational interface with limited memory
2. Document-based Question Answering using RAG
3. Text-to-image generation with prompt engineering
4. Multi-agent task handling using a controller (Weather, SQL, Recommender)
5. Final technical report on system design, debugging, and improvements

## Setup

In [11]:
# Import libraries

# === Core Python Libraries ===
import os
import replicate
import sqlite3
import requests
from IPython.display import display, Markdown
from datetime import datetime, timedelta
from typing import TypedDict, Annotated

# === LangChain 1.0 - Agent Framework ===
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# === LangGraph 1.0 - State Management & Checkpointing ===
from langgraph.checkpoint.memory import MemorySaver

# === Document Processing & Vector Storage ===
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# === OpenAI API Exceptions ===
from openai import (APIConnectionError, APIError, RateLimitError, AuthenticationError)

In [2]:
# Retrieve API keys from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")
WEATHER_API_KEY = os.environ.get("WEATHER_API_KEY")

if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

if REPLICATE_API_TOKEN is None:
    raise ValueError("REPLICATE_API_TOKEN environment variable not set.")

if WEATHER_API_KEY is None:
    raise ValueError("WEATHER_API_KEY environment variable not set.")

In [5]:
# Prepare RAG system
# Download NUS staff code of conduct from: https://www.nus.edu.sg/docs/defaultsource/corporate-files/about/code-of-conduct-nus-staff.pdf
# Modify filepath to document's filepath (either the NUS staff Code of Conduct pdf document or other desired document)
filepath = 'code-of-conduct-nus-staff.pdf'

# File validation
if not os.path.exists(filepath):
    raise FileNotFoundError(f"File not found: {filepath}")
if not os.access(filepath, os.R_OK):
    raise PermissionError(f"Cannot read file: {filepath}")

# Load document
try:
    loader = PyMuPDFLoader(filepath, mode="single")
    documents = loader.load()
    if not documents:
        raise ValueError("No content extracted from document.")
    print(f"Successfully loaded document from filepath: {filepath}.")
except Exception as e:
    raise RuntimeError(f"Failed to load PDF: {e}")

# Split into chunks
try:
    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)
    if not chunks:
        raise ValueError("No chunks produced. Check document parsing.")
    print(f"Split into {len(chunks)} chunks.")
except Exception as e:
    raise RuntimeError(f"Text splitting failed: {e}")

# Create embeddings
try:
    embedding_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
except AuthenticationError:
    raise RuntimeError("Invalid OpenAI API key. Please check OPENAI_API_KEY environment variable.")
except Exception as e:
    raise RuntimeError(f"Failed to initialize embeddings: {e}")

# Embed and store
try:
    vector_store = Chroma.from_documents(documents=chunks, embedding=embedding_model)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    print("✅ RAG retriever successfully created.")
except RateLimitError:
    print("⚠️ OpenAI rate limit reached. Please retry later.")
    retriever = None
except APIError as e:
    print(f"⚠️ OpenAI API error during embedding: {e}")
    retriever = None
except Exception as e:
    print(f"❌ Failed to create vector store: {e}")
    retriever = None

Successfully loaded document from filepath: code-of-conduct-nus-staff.pdf.
Split into 20 chunks.
✅ RAG retriever successfully created.


In [6]:
# Create events database
def setup_database():
    """Initialize the events database with sample data."""
    conn = None
    try:
        # Attempt to connect to (or create) the database
        conn = sqlite3.connect('events.db')
        c = conn.cursor()

        # Create table if not exists
        c.execute('''
            CREATE TABLE IF NOT EXISTS events (
                id INTEGER PRIMARY KEY,
                name TEXT,
                type TEXT,  -- 'indoor' or 'outdoor'
                description TEXT,
                location TEXT,
                country TEXT,
                date TEXT
            )
        ''')

        today = datetime.now().date()
        def iso(days=0): return (today + timedelta(days=days)).isoformat()

        # Synthetic event data
        events = [
            ("Marina Bay Food Festival", "outdoor", "A celebration of local and international cuisine", "Marina Bay, Singapore", "Singapore", iso(0)),
            ("Orchard Mall Art Fair", "indoor", "Pop-up art and design exhibition", "ION Orchard, Singapore", "Singapore", iso(0)),
            ("Mumbai Music Street", "outdoor", "Live indie music performances", "Marine Drive, Mumbai", "India", iso(0)),
            ("Delhi Book Conclave", "indoor", "Writers and readers meet-up", "Pragati Maidan, New Delhi", "India", iso(0)),
            ("Bangkok Street Carnival", "outdoor", "Street performances and food stalls", "Siam Square, Bangkok", "Thailand", iso(0)),
            ("Thai Craft Showcase", "indoor", "Traditional Thai crafts and art", "Bangkok Art Center, Bangkok", "Thailand", iso(0)),
            ("Penang Heritage Walk", "outdoor", "Tour of George Town’s historic district", "George Town, Penang", "Malaysia", iso(0)),
            ("KL Coffee Expo", "indoor", "Coffee tasting and workshops", "KL Convention Centre, Kuala Lumpur", "Malaysia", iso(0)),
            ("Jakarta Film Screening", "indoor", "Indie film premieres", "Cinema XXI, Jakarta", "Indonesia", iso(0)),
            ("Bali Sunset Beach Fest", "outdoor", "Beach music and food event", "Canggu, Bali", "Indonesia", iso(0)),
            ("Hanoi Street Parade", "outdoor", "Music and cultural performances", "Old Quarter, Hanoi", "Vietnam", iso(0)),
            ("Hanoi Art Studio", "indoor", "Local artist exhibition", "French Quarter, Hanoi", "Vietnam", iso(0)),
            ("Manila Food Market", "outdoor", "Filipino cuisine and music", "Intramuros, Manila", "Philippines", iso(0)),
            ("Manila Tech Expo", "indoor", "Startup and innovation exhibition", "SMX Convention Center, Manila", "Philippines", iso(0)),
            ("Singapore Jazz Night", "indoor", "Regional jazz bands live", "Esplanade, Singapore", "Singapore", iso(1)),
            ("Singapore Botanic Fair", "outdoor", "Flower and plant exhibition", "Singapore Botanic Gardens, Singapore", "Singapore", iso(1)),
            ("Chennai Dance Gala", "indoor", "Classical Bharatanatyam showcase", "Music Academy, Chennai", "India", iso(1)),
            ("Goa Beach Fest", "outdoor", "Open-air music by the sea", "Baga Beach, Goa", "India", iso(1)),
            ("Bangkok Food Carnival", "outdoor", "Street food extravaganza", "Chatuchak Market, Bangkok", "Thailand", iso(1)),
            ("Bangkok Innovation Hub", "indoor", "Tech startups and product demos", "Siam Discovery, Bangkok", "Thailand", iso(1)),
        ]

        # Insert data safely
        c.executemany('''
            INSERT OR IGNORE INTO events (name, type, description, location, country, date)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', events)

        conn.commit()
        print("✅ Database setup completed successfully. Events table is ready.")

    except sqlite3.OperationalError as e:
        print(f"❌ Database operational error: {e}")
    except sqlite3.IntegrityError as e:
        print(f"❌ Data integrity error during insertion: {e}")
    except Exception as e:
        print(f"❌ Unexpected error during setup: {e}")
    finally:
        if conn:
            conn.close()

# Run setup
setup_database()

✅ Database setup completed successfully. Events table is ready.


## Implementation

In [7]:
@tool
def retrieve_documents(query: str) -> str:
    """Retrieves relevant documents based on a search query."""
    retrieved_docs = retriever.invoke(query)
    return "\n\n".join([doc.page_content for doc in retrieved_docs])

# Store pending image requests for approval
pending_image_requests = {}

@tool
def request_image_generation(prompt: str, seed: int = 42, steps: int = 30) -> str:
    """
    Request to generate an image using Replicate API.
    This will ask for user approval before actually generating the image (costs money).
    
    Args:
        prompt: Description of the image to generate
        seed: Random seed for reproducibility (default: 42)
        steps: Number of generation steps (default: 30)
    
    Returns:
        A message indicating approval is needed
    """
    import uuid
    request_id = str(uuid.uuid4())
    pending_image_requests[request_id] = {
        "prompt": prompt,
        "seed": seed,
        "steps": steps
    }
    return f"🖼️ Image generation requested for: '{prompt}'\n\n⚠️ This will cost money via Replicate API. Please approve by calling 'approve_image_generation' with request_id: {request_id}"

@tool
def approve_image_generation(request_id: str) -> str:
    """
    Approve and execute a pending image generation request.
    
    Args:
        request_id: The ID of the pending image request to approve
    
    Returns:
        URL of the generated image or error message
    """
    if request_id not in pending_image_requests:
        return "⚠️ Invalid or expired request ID. No pending image generation found."
    
    request = pending_image_requests.pop(request_id)
    prompt = request["prompt"]
    seed = request["seed"]
    steps = request["steps"]
    
    try:
        output = replicate.run(
            "stability-ai/stable-diffusion-3.5-medium",
            input={"prompt": prompt, "seed": seed, "steps": steps}
        )

        # Handle unexpected response formats
        if isinstance(output, list):
            return output[0] if output else "⚠️ No image generated."
        elif hasattr(output, "url"):
            return output.url
        else:
            return str(output)
    except replicate.exceptions.ModelError as e:
        return f"⚠️ Image generation model error: {e}"
    except replicate.exceptions.ReplicateError as e:
        return f"⚠️ Replicate API error: {e}"
    except Exception as e:
        return f"⚠️ Unexpected image generation error: {e}"

@tool
def get_current_date() -> str:
    """
    Returns today's date in ISO format (YYYY-MM-DD).
    Use this tool when you need to know the current date for querying events or making date-based recommendations.
    """
    return datetime.now().date().isoformat()

In [8]:
@tool
def get_weather(location: str = 'Singapore') -> str:
    """
    Retrieve real-time weather data via the WeatherAPI.
    Takes a location as input and returns weather information including temperature and conditions.
    """
    url = "http://api.weatherapi.com/v1/current.json"
    params = {"key": WEATHER_API_KEY, "q": location, "aqi": "no"}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # Extract relevant weather information
        location_name = data['location']['name']
        country = data['location']['country']
        temp_c = data['current']['temp_c']
        condition = data['current']['condition']['text']
        
        return f"Weather in {location_name}, {country}: {temp_c}°C, {condition}"
    except requests.exceptions.Timeout:
        return f"⚠️ Weather API request timed out for {location}."
    except requests.exceptions.ConnectionError:
        return f"⚠️ Network connection failed while fetching weather for {location}."
    except requests.exceptions.HTTPError as e:
        return f"⚠️ Weather API HTTP error: {e}"
    except KeyError:
        return f"⚠️ Unexpected weather data format for {location}."
    except Exception as e:
        return f"⚠️ Unexpected weather retrieval error: {e}"

In [9]:
@tool
def get_events(date: str, event_type: str | None = None, country: str = 'Singapore') -> str:
    """
    Retrieves event data by querying the SQLite database for events on a given date.
    Optionally filters by event_type (indoor/outdoor) and country (default: Singapore).
    Returns a formatted string of matching events.
    """
    conn = None
    try:
        conn = sqlite3.connect('events.db')
        c = conn.cursor()
        
        if event_type:
            c.execute('SELECT * FROM events WHERE date=? AND type=? AND country=?', (date, event_type, country))
        else:
            c.execute('SELECT * FROM events WHERE date=? AND country=?', (date, country))
        
        events = c.fetchall()
        
        if not events:
            return f"No events found in {country} on {date}" + (f" ({event_type} type)" if event_type else " (all types)")
        
        # Format events nicely
        formatted_events = []
        for event in events:
            event_id, name, etype, desc, location, ecountry, edate = event
            formatted_events.append(
                f"- {name} ({etype}): {desc}. Location: {location}. Date: {edate}"
            )
        
        return "\n".join(formatted_events)
    
    except sqlite3.OperationalError as e:
        return f"⚠️ Database operational error: {e}"
    except sqlite3.DatabaseError as e:
        return f"⚠️ Database integrity error: {e}"
    except Exception as e:
        return f"⚠️ Unexpected database error: {e}"
    finally:
        if conn:
            try:
                conn.close()
            except:
                pass

In [12]:
# Recommendation Tool - Simple LLM Chain

from langchain_core.prompts import ChatPromptTemplate

# Create prompt template for recommendations
recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful event recommender. Consider the weather conditions and suggest suitable events. 
    For outdoor events, consider the temperature and weather conditions. Be specific about why you recommend certain events over others. 
    Keep your response concise but informative. 
    If event data is unavailable, politely request the user for additional event-related information.
    If weather data is unavailable, provide a balanced mix of indoor and outdoor suggestions."""),
    ("user", "{weather_and_event_data}")
])

# Create a simple chain: prompt → LLM
recommendation_chain = recommendation_prompt | ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

@tool
def recommend_events(weather_and_event_data: str) -> str:
    """
    Synthesize weather and event data into context-aware event recommendations.
    Takes combined weather and event information as input and returns personalized suggestions.
    
    Args:
        weather_and_event_data: Combined string containing weather conditions and available events
    
    Returns:
        A concise, personalized recommendation based on weather and events
    """
    try:
        # Invoke the chain with the input data
        result = recommendation_chain.invoke({"weather_and_event_data": weather_and_event_data})
        return result.content
    except Exception as e:
        return f"⚠️ Recommendation failed: {e}"

In [23]:
# Main Agent Setup - LangChain 1.0

# Define tools available to main agent
tools = [
    retrieve_documents,
    request_image_generation,
    approve_image_generation,
    get_current_date,
    get_weather,
    get_events,
    recommend_events
]

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

# Create checkpointer for conversation persistence
checkpointer = MemorySaver()

# System prompt for the main agent
system_prompt = """You are a helpful assistant with access to multiple tools.

**Tool Usage Guidelines:**
1. Use 'retrieve_documents' to find relevant information from the uploaded PDF before answering questions.
2. Use 'request_image_generation' when the user asks for image generation - this will request approval first.
3. After the user approves, use 'approve_image_generation' with the provided request_id to actually generate the image.
4. Use 'get_current_date' when you need to know today's date for event queries or recommendations.
5. Use 'get_weather' to retrieve weather information for a specific location (default: Singapore).
6. Use 'get_events' to query events from the database by date, type (indoor/outdoor), and country.
7. Use 'recommend_events' to synthesize weather and event data into personalized recommendations.

**Multi-Step Recommendation Workflow:**
When the user asks for event recommendations:
1. First, call 'get_current_date' to know today's date
2. Call 'get_weather' to get weather conditions for the location
3. Call 'get_events' to retrieve available events for the date
4. Finally, call 'recommend_events' with the combined weather and event data to generate personalized suggestions

Be helpful, concise, and informative in your responses."""

# Create the main agent using LangChain 1.0's create_agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

print("✅ Main agent created successfully with LangChain 1.0")

✅ Main agent created successfully with LangChain 1.0


In [24]:
# Chat Interface Functions - Updated for LangChain/LangGraph 1.0

# Thread configuration for conversation persistence
config = {"configurable": {"thread_id": "main_conversation"}}

def chat(user_input: str):
    """Send a message to the agent and display the response."""
    try:
        # Invoke agent with LangGraph 1.0 interface
        result = agent.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config=config
        )
        
        # Extract the last AI message
        if result and "messages" in result:
            messages = result["messages"]
            # Get the last assistant message
            ai_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if ai_messages:
                output = ai_messages[-1].content
                print("AI:")
                display(Markdown(output))
                return {"output": output, "messages": messages}
            else:
                print("⚠️ No response from agent.")
                return {"output": "⚠️ No response generated.", "messages": messages}
        else:
            print("⚠️ Unexpected response format.")
            return {"output": "⚠️ Unexpected response format.", "messages": []}
    
    except RateLimitError:
        print("⚠️ OpenAI rate limit reached. Try again shortly.")
        return {"output": "⚠️ Too many requests — please slow down."}
    except APIError as e:
        print(f"⚠️ OpenAI API error: {e}")
        return {"output": f"⚠️ The model encountered an API issue: {e}"}
    except AuthenticationError:
        print("❌ Invalid OpenAI credentials.")
        return {"output": "❌ Invalid OpenAI API key. Please verify configuration."}
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        return {"output": f"⚠️ Unexpected error occurred: {e}"}

def chat_loop():
    """Start an interactive chat session."""
    print("Chat started! Type 'quit' to exit.\n")
    print("💡 Conversation history is preserved across messages within this session.\n")
    while True:
        try:
            user_input = input("You: ").strip()
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("Goodbye!")
                break
            if not user_input:
                continue
            print(f"You: {user_input}")
            chat(user_input)
        except KeyboardInterrupt:
            print("\n🛑 Session interrupted by user.")
            break
        except Exception as e:
            print(f"⚠️ Error during chat loop: {e}")
            continue

def reset_conversation():
    """Reset the conversation by creating a new thread."""
    global config
    import uuid
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    print("✅ Conversation history cleared. Starting fresh.")

## Testing

In [25]:
chat_loop()

Chat started! Type 'quit' to exit.

💡 Conversation history is preserved across messages within this session.

You: How should NUS staff handle gifts?
AI:


NUS staff must handle gifts in accordance with relevant university policy documents. Specifically, the acceptance and provision of gifts, meals, and hospitality by staff is only permitted in line with these policies:

- The OFN Policy on Acceptance of Gifts and Hospitality by Staff
- Sponsorship by Industry Policy
- Business Meals and Employee-Related Functions Policy

These guidelines ensure that staff do not draw any personal gain or benefits from university-related business activities beyond their salary and standard compensation. Therefore, staff are expected to adhere to these policies to maintain ethical standards and avoid conflicts of interest.

You: Image of a white siamese cat
AI:


I can generate an image of a white Siamese cat for you. Please approve the request so I can proceed with the generation.

You: ok
AI:


Here is the generated image of a white Siamese cat:

![White Siamese Cat](https://replicate.delivery/xezq/Fpcnst4WGYZWFVSTVfdp6lRfoYGmSt5Mn27bQG7eGDK95bKrA/tmp4nqfqy7s.webp)

You: Recommend some events happening today
AI:


Here are some event recommendations for today:

1. **Marina Bay Food Festival** (Outdoor): Enjoy a celebration of local and international cuisine at Marina Bay. With the weather being pleasantly 26.3°C and partly cloudy, it's a great day to enjoy the outdoors and indulge in good food.

2. **Orchard Mall Art Fair** (Indoor): If you prefer indoor activities or have an interest in art, explore the pop-up art and design exhibition at ION Orchard. This indoor venue provides a comfortable setting to appreciate various artworks.

Choose based on your preferences for food and outdoor enjoyment, or art and a relaxed indoor environment!

Goodbye!


### Reference (legacy)

In [14]:
# Conversational interface with limited memory
# Document-based Question Answering using RAG

chat_loop()

Chat started! Type 'quit' to exit.



You:  How should NUS staff handle gifts?


AI: 


NUS staff must adhere to specific policies when handling gifts. The acceptance and provision of gifts, meals, and hospitality are allowed only in accordance with NUS policy documents, which include:

1. **OFN Policy on Acceptance of Gifts and Hospitality by Staff**: This policy outlines the rules for accepting gifts and hospitality to avoid conflicts of interest and maintain integrity.

2. **Sponsorship by Industry Policy**: This policy governs how sponsorships should be handled, ensuring transparency and fairness.

3. **Business Meals and Employee-Related Functions Policy**: This covers the acceptable conduct regarding business meals and related activities.

Staff should not derive any personal gain beyond their salary and employment terms from any business undertaken on behalf of the University. Compliance with these policies ensures ethical standards and helps in maintaining the integrity of the University's operations.

You:  What about if it is from a student?


...

You:  And what if the staff is also in a personal relationship with the student?


AI: 


In situations where a staff member has a personal relationship with a student, the handling of gifts becomes even more sensitive. Here are some guidelines that NUS staff should follow in such cases:

1. **Disclosure**: The staff member should disclose the personal relationship to their supervisor or relevant university office to maintain transparency and avoid any potential conflicts of interest.

2. **Avoid Conflicts of Interest**: The staff member should ensure that their personal relationship does not influence their professional responsibilities. They should recuse themselves from any academic or administrative decisions involving the student.

3. **Gifts Policy Adherence**: The acceptance of gifts should still adhere to the university's policies. If a gift is exchanged as part of the personal relationship, it should not be related to the staff member’s professional role or duties within the university.

4. **Professional Boundaries**: It is crucial for staff to maintain professional boundaries and ensure that the personal relationship does not affect their professional responsibilities or the student’s academic progress.

5. **Consultation and Guidance**: Staff members should seek guidance from their department or the appropriate university office if they are unsure of how to navigate the situation while complying with university policies.

By adhering to these principles, staff members can manage the potential risks associated with personal relationships with students and maintain ethical standards.

You:  quit


Goodbye!


In [25]:
# Text-to-image generation with prompt engineering

chat_loop()

Chat started! Type 'quit' to exit.



You:  Image of a white siamese cat


AI:


Here is an image of a white Siamese cat:

![White Siamese Cat](https://replicate.delivery/xezq/13xSdaao6WqSDxIeqJX0sRS1rUEKWPYpXQ06SKwxjPP6CTwKA/tmpu2shrd04.webp)

You:  Now show the results from these improved prompts: "a white siamese cat, studio lighting",  "a white siamese cat, dramatic lighting, dark background",  "a white siamese cat, outdoor setting, natural light"


AI:


Here are the images of a white Siamese cat with different lighting and settings:

1. **Studio Lighting**:
   ![White Siamese Cat, Studio Lighting](https://replicate.delivery/xezq/UUyadShxv3YBDFakR3jm3Da1vQsef1Kr7FcjOCqqfCkdMMBrA/tmpkyrjatek.webp)

2. **Dramatic Lighting, Dark Background**:
   ![White Siamese Cat, Dramatic Lighting, Dark Background](https://replicate.delivery/xezq/rBqJY6rLqbJPLZI6vSWCUuYy3648E0PtbftLYlr9ExwJDTwKA/tmpdkht_jw9.webp)

3. **Outdoor Setting, Natural Light**:
   ![White Siamese Cat, Outdoor Setting, Natural Light](https://replicate.delivery/xezq/p3QYKt4jorIjClVqUOeZAFJrmiFkewgavQwCAzPuHfOwMMBrA/tmpa7ag8qtx.webp)

You:  quit


Goodbye!


In [26]:
# Multi-agent task handling using a controller (Weather, SQL, Recommender)

chat_loop()

Chat started! Type 'quit' to exit.



You:  Recommend some events happening today


AI:


Here are some event recommendations for today in Singapore:

1. **Orchard Mall Art Fair**:
   - **Type**: Indoor
   - **Description**: Pop-up art and design exhibition.
   - **Location**: ION Orchard, Singapore
   - **Why Attend**: With the high temperature and humidity making it feel like 39.7°C (103.5°F), an indoor event like this is a comfortable choice. You can enjoy the art and design exhibition in a climate-controlled environment, avoiding the heat and humidity outside.

2. **Marina Bay Food Festival**:
   - **Type**: Outdoor
   - **Description**: A celebration of local and international cuisine.
   - **Location**: Marina Bay, Singapore
   - **Why Attend**: Although it's an outdoor event, the partly cloudy skies and low UV index mean you won't be exposed to intense sunlight, and the slight breeze might offer some relief. However, be prepared for the warm and humid conditions.

Both events offer unique experiences, so consider your comfort with the weather when making your choice.

You:  What about outdoor events in India?


...

You:  Would you recommend this based on the weather in Mumbai?


AI:


The current weather in Mumbai, India, is misty with a temperature of 34.2°C (93.6°F), but it feels like 42.9°C (109.2°F) due to the humidity. The UV index is moderate at 3.3, and there is a light breeze from the west at 7.4 mph (11.9 kph).

Given these conditions, attending the "Mumbai Music Street" outdoor event at Marine Drive could be enjoyable if you are prepared for the warm and humid weather. Make sure to stay hydrated, wear light clothing, and perhaps bring a fan or umbrella for shade. The misty conditions might add a unique atmosphere to the event, but visibility is somewhat limited at 3.0 km.

You:  quit


Goodbye!
